In [1]:
import pandas as pd
import numpy as np

# TODO: treat catch trials
#  convert this matlab code to python list format
filenames = ['interleaved_incomplete',
             'block_incomplete',
             'block_complete',
             'block_complete_simple',
             'block_complete_mixed',
             'block_complete_mixed_2s',
             'block_complete_mixed_2s_amb_heuristic',
             'block_complete_mixed_2s_amb',
             'incentivize']

# for f in filenames:
# print(f)
# df1 = pd.read_csv('data/behavior/csv/' + f + '.csv')

# ----------------------------------------------
#  This code is used to add prolific id to the data
# if f == 'block_complete_mixed':
#     df2 = pd.read_csv('data/behavior/with_prolific_id/' + f + '.csv')
#     df2.columns=["F"+str(i) for i in range(1, df2.shape[1]+1)]
#     break
#  rename columns
# for sub_id in df1['sub'].unique():
#     if np.isnan(sub_id):
#         continue


#     df1.loc[df1['sub'] == sub_id, 'prolific_id'] = df2['F3'][df2['F2'] == sub_id].values[0]
# ----------------------------------------------

for f in filenames:
    df = pd.read_csv(f'data/behavior/csv/{f}.csv')

    df.drop(['Unnamed: 0',
             'prolific_id',
             'cont1', 'cont2',
             'dbtime', '10',
             '26', '15', '16',
             'dist', 'plot'], axis=1, inplace=True)

    df.rename(columns={'17': 'chose_right', 'elic': 'phase',
              'sub': 'sub_id', '25': 'reversed'}, inplace=True)

    count = 0
    for sub in df.sub_id.unique():

        if np.isnan(sub):
            # drop nan sub
            df.drop(df[df.sub_id == sub].index, inplace=True)

        LE = (df.phase == -1) & (df.cond != -1) & (df.sub_id == sub)
        # count += sum(LE) == 120
        df.loc[LE, 'phase'] = 'LE'
        df.loc[LE, 'op1'] = 'E'
        df.loc[LE, 'op2'] = 'E'

        ES = (df.phase == 0) & (df.op2.isin(
            [0, -1])) & (df.op1.isin([1, -1])) & (df.sub_id == sub)
        # assert len(LE[ES]) == 0
        assert all(LE[ES] == False)
        # print(sum(ES))
        df.loc[ES, 'phase'] = 'ES'
        df.loc[ES, 'op1'] = 'E'
        df.loc[ES, 'op2'] = 'S'

        SP = (df.phase == 2) & (df.sub_id == sub)
        df.loc[SP, 'phase'] = 'SP'

        if f in ('block_complete_mixed',
                 'block_complete_mixed_2s',
                 'block_complete_mixed_2s_amb_heuristic',
                 'block_complete_mixed_2s_amb',
                 'incentivize'):
            EE = (df.phase == 0) & (df.op2 == 1) & (df.sub_id == sub)
            assert all((EE[ES] == False))
            df.loc[EE, 'phase'] = 'EE'
            df.loc[EE, 'op1'] = 'E'
            df.loc[EE, 'op2'] = 'E'

        if 'amb' in f:
            EA = (df.phase == 0) & (df.op2 == 2) & (
                df.op1 == 1) & (df.sub_id == sub)
            df.loc[EA, 'phase'] = 'EA'

            df.loc[EA, 'op1'] = 'E'
            df.loc[EA, 'op2'] = 'A'

            SA = (df.phase == 0) & (df.op2 == 2) & (
                df.op1 == 0) & (df.sub_id == sub)

            df.loc[SA, 'phase'] = 'SA'
            df.loc[SA, 'op1'] = 'S'
            df.loc[SA, 'op2'] = 'A'

    df.to_csv(f'data/behavior/reformat/{f}.csv')

    # TODO: treat catch trials
    # TODO: clean conditions
    # TODO: clean left_right


In [37]:
df = pd.read_csv('./data/behavior/reformat/block_complete.csv')
where = df.groupby('sub_id').phase.count().isin([288, 258])
ids = where.index[where.values]
df = df[df.sub_id.isin(ids)]
df = df[df.sess == 0]
df2 = df.groupby(['sub_id', 'phase', 'cond'], as_index=False)['corr'].mean()

df2 = df2[df2.phase=='LE']
import seaborn as sns

sns.barplot(data=df2, x='cond', y='corr')